In [1]:
# Create concepts using wikipedia (and its knowledge graph) 
# https://en.wikipedia.org/wiki/List_of_sports

# Search of sports: Found a link "List of sports"

In [44]:
import pandas as pd
import pyarrow as pa
from datasets import load_from_disk, Dataset
import os

In [45]:
path_dir = '/work3/s174498/concept_random_dataset/wikipedia_20220301/train'
wikipedia = load_from_disk(path_dir)

titles = wikipedia['title']

OSError: Memory mapping file failed: Cannot allocate memory

### Create concepts (Level II and Level III)

In [46]:
def main_articles(list_of_interest, folder_name):

    count_datasets = 0
    concept_list = []
    concept_list_saved = []
    concept_list_notsaved = []

    for i in range(len(list_of_interest)): 
        if list_of_interest[i] in titles:
            concept_list.append(list_of_interest[i])
            idx = titles.index(list_of_interest[i])

            # Clean up data (\n references in wiki article)
            text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
            text_clean2 = text_clean.replace('\n', ' ')
            phrase_to_list = text_clean2.split(". ")
            print(list_of_interest[i])
            print(len(phrase_to_list))

            if len(phrase_to_list) > 150:
                count_datasets = count_datasets + 1
                concept_list_saved.append(list_of_interest[i])

                df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
                'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
                data_concept = Dataset(pa.Table.from_pandas(df))

                # save data
                data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/' + folder_name + list_of_interest[i])
            else: 
                concept_list_notsaved.append(list_of_interest[i])
        else: 
            print(list_of_interest[i] + ' does not exist')

def sub_articles(list_of_interest, folder_name):
    count_datasets = 0
    concept_list = []
    concept_list_saved = []
    concatenate_phrases = []

    for i in range(len(list_of_interest)): 
        if list_of_interest[i] in titles:
            concept_list.append(list_of_interest[i])
            idx = titles.index(list_of_interest[i])

            # Clean up data (\n references in wiki article)
            text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
            text_clean2 = text_clean.replace('\n', ' ')
            phrase_to_list = text_clean2.split(". ")

            # Save concepts to "Level II"
            concatenate_phrases = concatenate_phrases + phrase_to_list

            # Save concepts to "Level III"
            if len(phrase_to_list) > 150:
                print(list_of_interest[i])
                concept_list_saved.append(list_of_interest[i])

                df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
                'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
                data_concept = Dataset(pa.Table.from_pandas(df))

                # save data
                path = os.path.join('/work3/s174498/concept_random_dataset/wikipedia_20220301/', folder_name)
                data_concept.save_to_disk(path + '/' + list_of_interest[i])

    if len(concatenate_phrases) > 150:
        df_levelII = pd.DataFrame({'title': folder_name, 'text_list': concatenate_phrases})
        data_concept_levelII = Dataset(pa.Table.from_pandas(df_levelII))

        # save data
        data_concept_levelII.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/' + folder_name)

### Create function to create concepts (level I)

In [ ]:
def create_concept_levelI(path_of_concepts, folder_name):

    list_of_folders = [os.path.join(path_of_concepts, o) for o in os.listdir(path_of_concepts) 
                        if os.path.isdir(os.path.join(path_of_concepts,o))]

    concat_text = []
    for i in range(len(list_of_folders)):
        data = load_from_disk(list_of_folders[i])
        data_text = data['text_list']

        concat_text = concat_text + data_text
        
        df_levelI = pd.DataFrame({'title': folder_name, 'text_list': concat_text})
        data_concept_levelI = Dataset(pa.Table.from_pandas(df_levelI))

        # save data
        data_concept_levelI.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/' + folder_name)

# Create datasets for sports concepts

Acrobatic sports: No 'Main articles'. Thus, all sub-articles are concatenated into one Level II concept, while these sub-articles (if longer than 150 sentences) are saved for Level III concepts.

In [47]:
acrobatic_sports = ['Ballet',
'Breakdancing',
'Competitive dancing',
'Cheerleading',
'Dancesport',
'Dragon dance',
'Figure skating',
'Freerunning',
'Gymnastics',
'High kick',
'Parkour',
'Pole sports',
'Stunt',
'Trampolining',
'Winter guard']

In [48]:
folder_name = 'sports_concepts/Acrobatic sports'
sub_articles(acrobatic_sports, folder_name)

Ballet
Breakdancing
Cheerleading
Figure skating
Gymnastics
Parkour


Next, try all conceps having a 'main articles'

In [49]:
list_of_sport = ['Air sports',
'Archery', 'Boardsport']

In [50]:
folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Air sports
5
Archery
322
Boardsport
71


Air sports: Not enought sentences in 'Main articles'. Thus, all sub-articles are concatenated into one Level II concept, while these sub-articles (if longer than 150 sentences) are saved for Level III concepts.

In [51]:
Air_sports = ['Aerobatics',
'Air racing',
'Gliding',
'Hang gliding',
'Human powered aircraft',
'Model aircraft',
'Parachuting',
'Paragliding',
'Paramotoring',
'Ultralight aviation']

In [52]:
folder_name = 'sports_concepts/Air sports'
sub_articles(Air_sports, folder_name)

Gliding
Hang gliding
Model aircraft
Parachuting
Paragliding


In [53]:
Boardsport = ['Skateboarding',
'Scootering',
'Skysurfing',
'Streetluge',
'Snowboarding',
'Surfing',
'Wakeboarding',
'Paddleboarding']

In [54]:
folder_name = 'sports_concepts/Boardsport'
sub_articles(Boardsport, folder_name)

Skateboarding
Snowboarding
Surfing


In [55]:
# no main article
Catching_games = ['Dodgeball',
'Frisbee',
'Gaga',
'Keep away',
'Kin-Ball',
'Newcomb ball',
'Quidditch',
'Yukigassen']

folder_name = 'sports_concepts/Catching games'
sub_articles(Catching_games, folder_name)

In [56]:
list_of_sport = ['Climbing', 'Cycle sport', 'Combat sport']
folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Climbing
46
Cycle sport
57
Combat sport
68


In [57]:
Climbing = ['Abseiling', 'Aid climbing',
'Ice climbing',
'Mixed climbing',
'Mountaineering',
'Rock climbing',
'Bouldering',
'Deep-water soloing',
'Sport climbing',
'Traditional climbing',
'Speed climbing',
'Sawanobori']

folder_name = 'sports_concepts/Climbing'
sub_articles(Climbing, folder_name)

Mountaineering
Rock climbing


In [58]:
Cycling = ['Bicycle', 'Skibobbing', 'Unicycle']

folder_name = 'sports_concepts/Cycling'
sub_articles(Cycling, folder_name)

Bicycle


In [59]:
Combat_sports = ['Aiki-jūjutsu', 'Jujutsu',
'Sambo (martial art)',
'Sumo'
'Wrestling', 
'Bajiquan'
'Bokator'
'Boxing'
'Capoeira'
'Chess boxing'
'Choi Kwang-Do'
'Fujian White Crane'
'Karate'
'Kenpō'
'Kickboxing'
'Lethwei'
'Muay Thai'
'Pradal serey'
'Savate'
'Shaolin Kung Fu'
'Shin-kicking'
'Sikaran'
'Silat'
'Subak'
'Taekkyeon'
'Taekwondo'
'Taidō'
'Tang Soo Do'
'Vovinam'
'Wing Chun'
'Bare knuckle boxing'
]

folder_name = 'sports_concepts/Combat sports'
sub_articles(Combat_sports, folder_name)

Jujutsu


In [60]:
Dog_sport = ['Herding',
'Sheepdog trial',
'Treibball',
'Heelwork to music',
'Musical canine freestyle',
'Obedience trial',
'Rally obedience',
'Schutzhund',
'Bikejoring',
'Canicross',
'Carting',
'Dog scootering',
'Mushing',
'Skijoring',
'Weight pulling',
'Dachshund racing',
'Greyhound and Whippet racing',
'Jack Russell Terrier',
'Sighthound',
'Sled dog racing',
'Terrier',
'Coon hunting field trial',
'Earthdog trial',
'Field trial',
'Hare coursing',
'Hound trailing',
'Lure coursing',
'Nosework',
'Tracking trial',
'Trail hunting',
'Agility',
'Disc dog',
'Flyball',
'Dog Puller']

folder_name = 'sports_concepts/Dog sports'
sub_articles(Dog_sport, folder_name)

Obedience trial
Jack Russell Terrier


In [61]:
Equestrian_sports = ['Buzkashi',
'Barrel racing',
'Campdrafting',
'Cirit',
'Charreada',
'Chilean rodeo',
'Chuckwagon racing',
'Cross country',
'Cutting',
'Dressage',
'Endurance riding',
'English pleasure',
'Equitation',
'Eventing',
'Equestrian vaulting',
'Gymkhana',
'Harness racing',
'Hobby horse polo',
'Horse racing',
'Horse polo',
'Horseball',
'Jineteada gaucha',
'Jousting',
'Mounted games',
'Mounted archery',
'Pato',
'Pleasure driving',
'Reining',
'Rodeo',
'Show hunter (British)',
'Show jumping',
'Steeplechase',
'Sur-papakh',
'Team penning',
'Tent pegging',
'Western pleasure']

folder_name = 'sports_concepts/Equestrian sports'
sub_articles(Equestrian_sports, folder_name)

Charreada
Dressage
Eventing
Harness racing
Horse racing
Mounted archery
Rodeo
Show jumping


In [62]:
list_of_sport = ['Fishing', 'Flying disc sports', 'Gymnastics']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Fishing
204
Flying disc sports
93
Gymnastics
314


In [63]:
Flying_disc_sports = ['Disc dog',
'Disc golf',
'Disc golf (urban)',
'Dodge disc',
'Double disc court',
'Freestyle',
'Frisbee',
'Goaltimate',
'KanJam',
'Guts',
'Hot box',
'Ultimate']

folder_name = 'sports_concepts/Flying disc sports'
sub_articles(Flying_disc_sports, folder_name)

Disc golf


In [64]:
list_of_sport = ['Competitive yoga', 'Hunting']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Competitive yoga
19
Hunting
452


In [65]:
Ice_sports = ['Bandy',
'Barrel jumping',
'Broomball',
'Curling',
'Figure skating',
'Ice cross downhill',
'Ice football',
'Ice hockey',
'Ice yachting',
'Moscow broomball',
'Ringette',
'Rinkball',
'Speed skating',
'Ice stock sport']

folder_name = 'sports_concepts/Ice sports'
sub_articles(Ice_sports, folder_name)

Bandy
Curling
Figure skating
Ice hockey
Ringette


In [66]:
Kite_sports = ['Hang gliding',
'Kite buggy',
'Kite fighting',
'Kite landboarding',
'Kitesurfing',
'Parasailing',
'Snow kiting',
'Sport kite']

folder_name = 'sports_concepts/Kite sports'
sub_articles(Kite_sports, folder_name)

Hang gliding


In [67]:
Mixed_discipline = ['Adventure racing',
'Aquathlon',
'Biathlon',
'Duathlon',
'Decathlon',
'Heptathlon',
'Icosathlon',
'Modern pentathlon',
'Pentathlon',
'Tetrathlon',
'Triathlon'
]

folder_name = 'sports_concepts/Mixed discipline'
sub_articles(Mixed_discipline, folder_name)

Triathlon


### Create 'Level I' concept: Sport

In [119]:
path_of_sports = '/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/'
folder_name = 'sports_concepts/'

create_concept_levelI(path_of_sports, folder_name)

In [120]:
# load_from_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/')

Dataset({
    features: ['title', 'text_list'],
    num_rows: 18757
})